In [28]:
import os
import scipy.io as scio
import shutil
from PIL import Image, ImageDraw

In [2]:
DATABASE_FOLDER = '../data/hatdb'
IMAGES_FOLDER = os.path.join(DATABASE_FOLDER, 'images')
ANNOTATION_FILE = os.path.join(DATABASE_FOLDER, 'anno.mat')

In [3]:
VOC_DATA = '../data/VOC0712'
VOC_ANNOTATIONS = os.path.join(VOC_DATA, 'Annotations')
VOC_IMAGESETS = os.path.join(os.path.join(VOC_DATA, 'ImageSets'), 'Main')
VOC_JPEGIMAGES = os.path.join(VOC_DATA, 'JPEGImages')

In [36]:
idx_number = 1
numb = idx_number

In [5]:
if not os.path.isdir(VOC_DATA):
    os.makedirs(VOC_DATA)
if not os.path.isdir(VOC_ANNOTATIONS):
    os.makedirs(VOC_ANNOTATIONS)
if not os.path.isdir(VOC_IMAGESETS):
    os.makedirs(VOC_IMAGESETS)
if not os.path.isdir(VOC_JPEGIMAGES):
    os.makedirs(VOC_JPEGIMAGES)

In [6]:
annotation_file = scio.loadmat(ANNOTATION_FILE)
annotation_file

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Apr 28 17:46:59 2012',
 '__version__': '1.0',
 'anno': array([[(array([[array(['q0_r10.jpg'], dtype='<U10'),
         array(['q0_r14.jpg'], dtype='<U10'),
         array(['q0_r15.jpg'], dtype='<U10'), ...,
         array(['q328_r46.jpg'], dtype='<U12'),
         array(['q328_r53.jpg'], dtype='<U12'),
         array(['q328_r5.jpg'], dtype='<U11')]], dtype=object), array([[  1,  28,  98, 298,   1],
        [  1,  75, 453, 500,   2],
        [241,  75, 479, 346,   2],
        ...,
        [  1,  15, 303, 348,   2],
        [  1,  51, 351, 375,   2],
        [ 13,  49, 337, 498,   1]], dtype=uint16), array([[1, 0, 1, ..., 1, 0, 0]], dtype=uint8), array([[ 1, -1, -1, ..., -1, -1,  1],
        [-1,  1,  1, ...,  1,  1, -1],
        [ 1, -1, -1, ..., -1, -1,  1],
        ...,
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1,  0,  0, ...,  0,  0,  0]], dtype=int

In [7]:
# лист файлов картинок базы, каждый элемент которого - массив с 1 элементом
img_files = annotation_file['anno']['files'][0][0][0].tolist()

In [8]:
# лист с bboxes для каждой картинки из базы, каждый элемент которого - лист (важны только первые 4 элемента листа)
bboxes = annotation_file['anno']['objbbs'][0][0].tolist()

In [9]:
# лист меток пола
# 1 - Ж, -1 - М, 0 - ?
sexes = annotation_file['anno']['y'][0][0][0].tolist()

In [10]:
def get_gender(label):
    if (label == 1):
        return 'f'
    if (label == -1):
        return 'm'
    if (label == 0):
        return 'u'
    return 'u'

In [11]:
voc_dict = dict()

In [12]:
for i in range(len(img_files)):
    gend = get_gender(sexes[i])
    if gend != 'u':
        pict = img_files[i][0]
        bbox = bboxes[i][:4]
        bbox[0] += 1
        bbox[1] += 1
        bbox[2] -= 1
        bbox[3] -= 1
        voc_content = [bbox, gend]
        if not (pict in voc_dict.keys()):
            voc_dict[pict] = [voc_content]
        else:
            voc_dict[pict].append(voc_content)

In [13]:
def xml_objects(objs):
    res = ""
    for el in objs:
        res += '  <object>\n' + \
        '    <name>{}</name>\n'.format(el[1]) + \
        '    <pose>Unspecified</pose>\n' + \
        '    <truncated>0</truncated>\n' + \
        '    <difficult>0</difficult>\n' + \
        '    <bndbox>\n' + \
        '      <xmin>{}</xmin>\n'.format(el[0][0]) + \
        '      <ymin>{}</ymin>\n'.format(el[0][1]) + \
        '      <xmax>{}</xmax>\n'.format(el[0][2]) + \
        '      <ymax>{}</ymax>\n'.format(el[0][3]) + \
        '    </bndbox>\n' + \
        '  </object>\n'
    return res

In [14]:
def xml_content(file_name, voc_el):
    xml_objs = xml_objects(voc_el)
    if (xml_objs != ""):
        return '<?xml version="1.0" encoding="utf-8"?>\n' + \
                '<annotation>\n' + \
                '  <folder>VOC0712</folder>\n' + \
                '  <filename>{}</filename>\n'.format(file_name) + \
                '  <source>\n' + \
                '    <database>KAMAZ Database</database>\n' + \
                '    <annotation>KAMAZ</annotation>\n' + \
                '    <image>flickr</image>\n' + \
                '    <flickrid>0</flickrid>\n' + \
                '  </source>\n' + \
                '  <owner>\n' + \
                '    <flickrid>Unspecified</flickrid>\n' + \
                '    <name>Unspecified</name>\n' + \
                '  </owner>\n' + \
                '  <size>\n' + \
                '    <width>1920</width>\n' + \
                '    <height>1080</height>\n' + \
                '    <depth>3</depth>\n' + \
                '  </size>\n' + \
                '  <segmented>0</segmented>\n' + xml_objs + '</annotation>'

In [15]:
def create_marking_xml(file_name, voc_el):
    with open(os.path.join(VOC_ANNOTATIONS, file_name+ '.xml'), 'w') as output_file:
        str_data = xml_content(file_name + '.jpg', voc_el)
        output_file.write(str_data)

In [16]:
list_files = []
for k in voc_dict:
    voc_el = voc_dict[k]
    file_name = '{:0>6}'.format(idx_number)
    list_files.append(file_name)
    shutil.copyfile(os.path.join(IMAGES_FOLDER, k), os.path.join(VOC_JPEGIMAGES, file_name + '.jpg'))
    create_marking_xml(file_name, voc_el)
    idx_number += 1

In [64]:
with open(os.path.join(VOC_IMAGESETS, 'hat_imagesets.txt'), 'w') as output_file:
    for el in list_files:
        output_file.write(el+'\n')